Install TMDBvAPI

In [ ]:
pip install tmdbv3api

In [ ]:
import requests
import json
import time
import pandas as pd
from tmdbv3api import TMDb, TV
from tmdbv3api.exceptions import TMDbException

Funtions to collect Metadata of Friends Episodes

In [ ]:
# Function to get the ID of the TV show
def get_show_id(api_key, query):
    url = f"https://api.themoviedb.org/3/search/tv?api_key={api_key}&query={query}"
    response = requests.get(url)
    data = response.json()
    if 'results' in data and data['results']:
        # Assuming the first show in the results is the one we're looking for
        return data['results'][0]['id']
    return None  # Handle no results or errors appropriately

# Function to get all seasons of the TV show
def get_seasons(api_key, show_id):
    url = f"https://api.themoviedb.org/3/tv/{show_id}?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'seasons' in data:
        return data['seasons']
    return None  # Handle no results or errors appropriately

# Function to get all episodes of a season
def get_episodes(api_key, show_id, season_number):
    url = f"https://api.themoviedb.org/3/tv/{show_id}/season/{season_number}?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'episodes' in data:
        return data['episodes']
    return None  # Handle no results or errors appropriately


def get_episode_details(episode):
    # Extract the basic episode details
    episode_details = {
        'id': episode['id'],
        'air_date': episode['air_date'],
        'episode_number': episode['episode_number'],
        'name': episode['name'],
        'overview': episode['overview'],
        'production_code': episode['production_code'],
        'runtime': episode.get('runtime', pd.NA),
        'season_number': episode['season_number'],
        'vote_average': episode['vote_average'],
        'vote_count': episode['vote_count']
    }

    # Extract crew details and add episode_id
    crew_details = [{
        'episode_id': episode['id'],
        'department': member.get('department', pd.NA),
        'job': member.get('job', pd.NA),
        'adult': member.get('adult', pd.NA),
        'gender': member.get('gender', pd.NA),
        'id': member.get('id', pd.NA),
        'name': member.get('name', pd.NA),
        'original_name': member.get('original_name', pd.NA),
        'popularity': member.get('popularity', pd.NA),
        'profile_path': member.get('profile_path', pd.NA)
    } for member in episode.get('crew', [])]

    # Extract guest stars details and add episode_id
    guest_stars_details = [{
        'episode_id': episode['id'],
        'character': star.get('character', pd.NA),
        'credit_id': star.get('credit_id', pd.NA),
        'order': star.get('order', pd.NA),
        'adult': star.get('adult', pd.NA),
        'gender': star.get('gender', pd.NA),
        'id': star.get('id', pd.NA),
        'name': star.get('name', pd.NA),
        'original_name': star.get('original_name', pd.NA),
        'popularity': star.get('popularity', pd.NA),
        'profile_path': star.get('profile_path', pd.NA)
    } for star in episode.get('guest_stars', [])]

    return episode_details, crew_details, guest_stars_details



In [ ]:
# Main function to collect data
def main():
    api_key = 'f082066c7a8bedc4aab8ca888b9cc562'  # Replace with your TMDb API key
    show_name = "Friends"

    # Step 1: Get the TV show ID
    show_id = get_show_id(api_key, show_name)
    if not show_id:
        print(f"Could not find a show with the name {show_name}")
        return

    # Step 2: Get all the seasons for the TV show
    seasons = get_seasons(api_key, show_id)
    if not seasons:
        print(f"Could not find seasons for the show {show_name}")
        return

    # Initialize DataFrames
    episode_columns = ['id','air_date', 'episode_number', 'name', 'overview', 'production_code',
                       'runtime', 'season_number', 'vote_average', 'vote_count']
    crew_columns = ['episode_id', 'department', 'job', 'adult', 'gender', 'id', 'name', 'original_name', 'popularity', 'profile_path']
    guest_stars_columns = ['episode_id', 'character', 'credit_id', 'order', 'adult', 'gender', 'id', 'name', 'original_name', 'popularity', 'profile_path']

    episode_df = pd.DataFrame(columns=episode_columns)
    crew_df = pd.DataFrame(columns=crew_columns)
    guest_stars_df = pd.DataFrame(columns=guest_stars_columns)

    # Step 3: For each season, get all episodes
    for season in seasons:
        season_number = season['season_number']
        episodes = get_episodes(api_key, show_id, season_number)
        if not episodes:
            print(f"Could not find episodes for season {season_number} of the show {show_name}")
            continue

        # Process each episode
        for episode_data in episodes:
            episode, crew, guest_stars = get_episode_details(episode_data)
            print(episode)
            episode['season_number'] = season_number  # Adding season number to episode data

            # Append episode data to episode_df
            episode_df = episode_df.append({col: episode[col] for col in episode_columns}, ignore_index=True)

            # Append crew data to crew_df
            for member in crew:
                member_data = {col: member.get(col) for col in crew_columns if col != 'episode_id'}
                member_data['episode_id'] = episode['id']
                crew_df = crew_df.append(member_data, ignore_index=True)

            # Append guest stars data to guest_stars_df
            for star in guest_stars:
                star_data = {col: star.get(col) for col in guest_stars_columns if col != 'episode_id'}
                star_data['episode_id'] = episode['id']
                guest_stars_df = guest_stars_df.append(star_data, ignore_index=True)

            # Handle rate limits and be respectful to the API server
            time.sleep(0.5)

    # Save the DataFrames to CSV files
    episode_df.to_csv('friends_episodes.csv', index=False)
    crew_df.to_csv('friends_crew.csv', index=False)
    guest_stars_df.to_csv('friends_guest_stars.csv', index=False)

# This will not run the main function because we are not executing the script,
# but you would typically include this in your Python script.
if __name__ == "__main__":
    main()


{'id': 2639442, 'air_date': '2001-02-15', 'episode_number': 1, 'name': "The Stuff You've Never Seen", 'overview': 'A 20-minute special hosted by Conan O’Brien featuring some of the cast’s favorite outtakes from past episodes.', 'production_code': '', 'runtime': 25, 'season_number': 0, 'vote_average': 10.0, 'vote_count': 1}


<ipython-input-41-ccffbf556c68>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append({col: episode[col] for col in episode_columns}, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  guest_stars_df = guest_stars_df.append(star_data, ignore_index=True)


{'id': 2639441, 'air_date': '2004-05-06', 'episode_number': 2, 'name': 'The One Before the Last One - Ten Years of Friends', 'overview': 'A collection of clips from past episodes, past seasons, leading to the final episode.', 'production_code': '', 'runtime': 25, 'season_number': 0, 'vote_average': 0.0, 'vote_count': 0}


<ipython-input-41-ccffbf556c68>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append({col: episode[col] for col in episode_columns}, ignore_index=True)


{'id': 1988539, 'air_date': None, 'episode_number': 3, 'name': 'Friends of Friends (Season 1)', 'overview': 'Video guestbook of the guest stars featured in Season 1: Hank Azaria, George Clooney & Noah Wyle, Morgan Fairchild & Jay Leno, Jill Goodarce, Elliot Gould, Jennifer Grey, Helen Hunt, Jon Lovitz, Christina Pickles, Leah Remini, Harry Shearer, Jonathan Silverman, Fisher Stevens, Brenda Vaccaro', 'production_code': '', 'runtime': 9, 'season_number': 0, 'vote_average': 0.0, 'vote_count': 0}


...

<ipython-input-41-ccffbf556c68>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append({col: episode[col] for col in episode_columns}, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipytho

{'id': 87633, 'air_date': '2004-04-29', 'episode_number': 16, 'name': "The One with Rachel's Going Away Party", 'overview': "With Rachel leaving tomorrow, the gang hosts a farewell party for her. She says goodbye to each of her friends individually, except for Ross and he's offended. Monica precisely packs for her new life in the suburbs with Chandler, and they keep a watchful eye on the extremely pregnant Erica, who is in town until the baby arrives.", 'production_code': '176265', 'runtime': 23, 'season_number': 10, 'vote_average': 8.661, 'vote_count': 29}


<ipython-input-41-ccffbf556c68>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append({col: episode[col] for col in episode_columns}, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipytho

{'id': 87632, 'air_date': '2004-05-06', 'episode_number': 17, 'name': 'The Last One', 'overview': "Ross and Phoebe chase Rachel to the airport, but end up at the wrong one. Chandler and Monica finish packing for their move to the suburbs, and Joey loses Chick Jr. and Duck Jr. in the foosball table.\n\nThe series finale finds Rachel, Monica, Phoebe, Joey, Chandler and Ross embarking on the next chapters in their lives. The six of them have been there for one another through all the ups and downs of becoming adults. Now it's their last day together, and it's one of momentous events and last-minute surprises. Even as the friends make major decisions, they share a bond that will last forever.", 'production_code': '176266 / 176267', 'runtime': 48, 'season_number': 10, 'vote_average': 9.4, 'vote_count': 33}


<ipython-input-41-ccffbf556c68>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append({col: episode[col] for col in episode_columns}, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipython-input-41-ccffbf556c68>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crew_df = crew_df.append(member_data, ignore_index=True)
<ipytho

In [ ]:
episodes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               267 non-null    int64  
 1   air_date         230 non-null    object 
 2   episode_number   267 non-null    int64  
 3   name             267 non-null    object 
 4   overview         266 non-null    object 
 5   production_code  228 non-null    object 
 6   runtime          267 non-null    int64  
 7   season_number    267 non-null    int64  
 8   vote_average     267 non-null    float64
 9   vote_count       267 non-null    int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 21.0+ KB


In [ ]:
episodes_df=episodes_df.drop(columns=['production_code','vote_count'])

In [ ]:
episodes_df

,id,air_date,episode_number,name,overview,runtime,season_number,vote_average
39,85987,1994-09-22,1,Pilot,An introduction to the gang. After Rachel leav...,23,1,7.081
40,86012,1994-09-29,2,The One with the Sonogram at the End,"Ross's lesbian ex-wife, Carol, is pregnant wit...",23,1,7.333
41,85981,1994-10-06,3,The One with the Thumb,Monica finds it difficult to break up with her...,23,1,8.130
42,85983,1994-10-13,4,The One with George Stephanopoulos,While the men attend a hockey game and end up ...,23,1,7.840
43,86034,1994-10-20,5,The One with the East German Laundry Detergent,While Chandler and Phoebe decide to break up w...,23,1,8.041
...,...,...,...,...,...,...,...,...
262,87603,2004-02-19,13,The One Where Joey Speaks French,Joey implores Francophile Phoebe to teach him ...,24,10,8.500
263,87607,2004-02-26,14,The One with Princess Consuela,"Monica and Chandler visit their house, which i...",24,10,8.700
264,87628,2004-04-22,15,The One Where Estelle Dies,"When Rachel makes a startling announcement, Ro...",24,10,8.800
265,87633,2004-04-29,16,The One with Rachel's Going Away Party,"With Rachel leaving tomorrow, the gang hosts a...",23,10,8.661


Collecting Crew data

In [ ]:
crew_df=crew_df.drop(columns=['adult','gender','id','name','profile_path','department'])
crew_df.rename(columns={'episode_id': 'id'}, inplace=True)
crew_df

,id,job,original_name,popularity
0,2639442,Director,Kevin S. Bright,1.519
1,85987,Writer,Marta Kauffman,5.777
2,85987,Writer,David Crane,3.760
3,85987,Director,James Burrows,5.803
4,85987,Executive Producer,David Crane,1.368
...,...,...,...,...
1126,87633,Writer,Andrew Reich,1.788
1127,87633,Writer,Ted Cohen,3.069
1128,87632,Director,Kevin S. Bright,1.519
1129,87632,Writer,Marta Kauffman,5.777


Collecting Guest Stars data

In [ ]:
guest_stars_df=guest_stars_df.drop(columns=['adult','gender','id','name','profile_path','credit_id','order','original_name'])
guest_stars_df.rename(columns={'episode_id': 'id'}, inplace=True)
guest_stars_df

,id,character,popularity
1,85987,Paul,10.864
2,85987,Franny,6.415
3,85987,Jasmine,5.117
4,86012,Carol Willick,6.936
5,86012,Dr. Oberman,5.236
...,...,...,...
1024,87628,Janice,19.494
1025,87633,Erica,29.614
1026,87632,Mike Hanningan,42.535
1027,87632,Erica,29.614


In [ ]:
characters_df = guest_stars_df.groupby('id').agg(
    list_of_characters=('character', list),
    average_character_popularity=('popularity', 'mean')
).reset_index()
characters_df


,id,list_of_characters,average_character_popularity
0,85979,"[Terry, Susan Bunch, Carol Willick, Gunther]",14.485000
1,85981,"[Lizzy, Paula, Alan, Gunther]",9.014250
2,85983,"[Nurse Sizemore, Joanne, Leslie, Pizza Guy, Br...",6.773571
3,85987,"[Paul, Franny, Jasmine]",7.465333
4,85990,"[Aunt Lillian, Lowell, Shelley, Jack Geller, J...",9.672667
...,...,...,...
218,1007806,"[Richard Burke, Ryan, Jeannie, Scott Alexander...",16.832400
219,1007807,"[Ginger, Leslie, Mark Robinson, Gunther]",16.264500
220,1007808,"[The Singing Man, Emily Waltham, Gunther]",7.928000
221,1007810,"[Judge, Porsche Owner, Gunther]",9.143333


In [ ]:
final_df = pd.merge(episodes_df, characters_df , on='id', how='inner')
final_df

,id,air_date,episode_number,name,overview,runtime,season_number,vote_average,list_of_characters,average_character_popularity
0,85987,1994-09-22,1,Pilot,An introduction to the gang. After Rachel leav...,23,1,7.081,"[Paul, Franny, Jasmine]",7.465333
1,86012,1994-09-29,2,The One with the Sonogram at the End,"Ross's lesbian ex-wife, Carol, is pregnant wit...",23,1,7.333,"[Carol Willick, Dr. Oberman, Robbie, Marsha, B...",10.371889
2,85981,1994-10-06,3,The One with the Thumb,Monica finds it difficult to break up with her...,23,1,8.130,"[Lizzy, Paula, Alan, Gunther]",9.014250
3,85983,1994-10-13,4,The One with George Stephanopoulos,While the men attend a hockey game and end up ...,23,1,7.840,"[Nurse Sizemore, Joanne, Leslie, Pizza Guy, Br...",6.773571
4,86034,1994-10-20,5,The One with the East German Laundry Detergent,While Chandler and Phoebe decide to break up w...,23,1,8.041,"[Horrible Woman, Angela, Bob, Janice, Gunther]",8.251000
...,...,...,...,...,...,...,...,...,...,...
218,87603,2004-02-19,13,The One Where Joey Speaks French,Joey implores Francophile Phoebe to teach him ...,24,10,8.500,"[Erica, Nurse, Leonard Green, Casting Assistan...",8.232600
219,87607,2004-02-26,14,The One with Princess Consuela,"Monica and Chandler visit their house, which i...",24,10,8.700,"[Mike Hanningan, Mackenzie, James Campbell, Cl...",17.321778
220,87628,2004-04-22,15,The One Where Estelle Dies,"When Rachel makes a startling announcement, Ro...",24,10,8.800,"[Ellen, Mr. Simon, Emma Geller-Green (uncredit...",11.355400
221,87633,2004-04-29,16,The One with Rachel's Going Away Party,"With Rachel leaving tomorrow, the gang hosts a...",23,10,8.661,[Erica],29.614000


In [ ]:
from google.colab import files
files.download('friends_episodes.csv')
files.download('friends_crew.csv')
files.download('friends_guest_stars.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>